<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/Transform2/Transformer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модель Transformer-2

## Доставляем библиотеки и импортируем их

In [1]:
!pip install transformers transformers[sentencepiece] sentencepiece datasets sentence-transformers rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import torch

from datasets import load_dataset

from tqdm import tqdm; tqdm.pandas()

from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GRU, AdditiveAttention

import nltk; nltk.download('punkt')
from nltk import sent_tokenize

import unicodedata
import re
import os
import io
import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Скачиваем данные и подготавливаем

In [3]:
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")
dataset

No config specified, defaulting to: gazeta/default
Reusing dataset gazeta (/root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 52400
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5770
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5265
    })
})

In [4]:
df_test = pd.DataFrame(dataset["test"])
df_train = pd.DataFrame(dataset["train"])
df_test.head()

,text,summary,title,date,url
0,Американское аэрокосмическое агентство NASA ог...,"В NASA назвали четыре миссии в дальний космос,...","Венера, Ио или Тритон: куда полетит NASA",2020-02-14 16:39:11,https://www.gazeta.ru/science/2020/02/14_a_129...
1,Около 11 тысяч зрителей увидели все самое лучш...,25 и 26 февраля в Кремлевском дворце съездов п...,«Люди в Бурятии очень талантливые»,2020-02-28 10:44:13,https://www.gazeta.ru/social/2020/02/28/129806...
2,7 ноября в Белоруссии прошли выборы членов сов...,В Белоруссии в день годовщины Октябрьской рево...,Вспомнить СССР: как Лукашенко провел выборы,2019-11-07 19:55:08,https://www.gazeta.ru/politics/2019/11/07_a_12...
3,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,2020-03-01 16:50:06,https://www.gazeta.ru/culture/2020/03/01/a_129...
4,Депутат Верховной рады от партии «Слуга народа...,Украина не должна выплачивать пенсии жителям Д...,«Поддерживают Россию»: почему Киев не платит п...,2020-02-06 12:41:24,https://www.gazeta.ru/business/2020/02/06/1294...


## Дополнительные функции

In [5]:
def calc_scores(references, predictions, metric="all"):
  """
    Функция подсчета метрик
  """
  print("Count:", len(predictions))
  print("Ref:", references[-1])
  print("Hyp:", predictions[-1])

  if metric in ("bleu", "all"):
      print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
  if metric in ("rouge", "all"):
      rouge = Rouge()
      scores = rouge.get_scores(predictions, references, avg=True)
      print("ROUGE: ", scores)

def summary_model(model_name, df_test):
  """
    Подсчет метрик на примере предсказания модели
  """
  print(f"Model: {model_name}")
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  article_text = df_test['text'][1]

  input_ids = tokenizer(
      [article_text],
      max_length=600,
      add_special_tokens=True,
      padding="max_length",
      truncation=True,
      return_tensors="pt"
  )["input_ids"]

  output_ids = model.generate(
      input_ids=input_ids,
      no_repeat_ngram_size=4
  )[0]

  summary = tokenizer.decode(output_ids, skip_special_tokens=True)
  print(f"summary: {summary}\n")

  target = sent_tokenize(df_test['summary'][1])
  preds = sent_tokenize(summary)

  calc_scores(target, preds)

def loss_function(real, pred):
    """
      Функция потери
    """
    # определяем ненулевые элементы
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    #переводим mask к типу loss_
    mask = tf.cast(mask, dtype=loss_.dtype)
    #умножаем loss на маску
    loss_ *= mask

    return tf.reduce_mean(loss_) #возвращаем среднее значение элементов тензора

@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
      enc_output, enc_hidden = encoder(inp, enc_hidden)

      dec_hidden = enc_hidden

      dec_input = tf.expand_dims([sum_wordindex['bos']] * BATCH_SIZE, 1)

      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
          # passing enc_output to the decoder
          predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

          loss += loss_function(targ[:, t], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

def evaluate(sentence, tok_text, tok_sum, max_len_text = 700, max_len_sum = 100):
  """
    Оценика модели
  """
  inputs = [tok_text.word_index[i] for i in sentence.split(' ') if i !='']
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_len_text,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, latent_dim))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([tok_sum.word_index['bos']], 0)

  for t in range(max_len_sum):
      predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

      predicted_id = tf.argmax(predictions[0]).numpy()

      result += tok_sum.index_word[predicted_id] + ' '

      if tok_sum.index_word[predicted_id] == 'eos':
          return result

      # the predicted ID is fed back into the model
      dec_input = tf.expand_dims([predicted_id], 0)

  return result

def seq2text(input_seq):
  """
    Функция перевода последовательности индексов в текст
  """
  newString = ''
  for i in input_seq:
      if i != 0:
          newString = newString + reverse_text_index[i] + ' '
  return newString

def seq2summary(input_seq):
  """
    Функция перевода последовательности индексов в title
  """
  newString = ''
  for i in input_seq:
      if (i != 0 and i != sum_wordindex['bos']) and i != sum_wordindex['eos']:
          newString = newString + reverse_sum_index[i] + ' '
  return newString

## Глобальные параметры

In [6]:
MAX_TRAIN_SAMPLE = 10000
MAX_TEST_SAMPLE = 2000

EPOCHS = 200

BATCH_SIZE = 64
BUFFER_SIZE = 1

## Дополнительные классы

In [7]:
class Transform_2():
  
  __token = None
  __text_vocab_size = None
  __seq_token = {
      "test": None,
      "train": None
  }
  __padded = {
      "test": None,
      "train": None
  }
  max_len = 0

  def __init__(self, max_len = 700):
    self.__token = Tokenizer()
    self.max_len = max_len

  def get_token(self):
    return self.__token
  
  def get_size(self):
    return self.__text_vocab_size
  
  def get_padded(self):
    return self.__padded
  
  def __eval_token(self, txt_train, txt_test):
    self.__token.fit_on_texts(txt_train)
    self.__seq_token['train'] = self.__token.texts_to_sequences(txt_train)
    self.__seq_token['test'] = self.__token.texts_to_sequences(txt_test)
  
  def transfrom_text(self, txt_train, txt_test):
    self.__eval_token(txt_train, txt_test)

    self.__text_vocab_size = len(self.__token.word_index) + 1
    self.__padded["train"] = pad_sequences(self.__seq_token['train'], maxlen=self.max_len, padding='post', truncating='post')
    self.__padded["test"] = pad_sequences(self.__seq_token['test'], maxlen=self.max_len, padding='post', truncating='post')  

## 1. Провериv насколько хорошо она суммаризирует модель *rut5_base_sum_gazeta*

In [8]:
models = ["IlyaGusev/mbart_ru_sum_gazeta", "IlyaGusev/rut5_base_sum_gazeta"]
for model in models:
  summary_model(model, df_test)

Model: IlyaGusev/mbart_ru_sum_gazeta
summary: В Кремле прошла премьера новогоднего шоу «Танцуют все!» с участием более 300 артистов из одного региона. Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков.

Count: 2
Ref: Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Hyp: Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков.
BLEU:  0.15606720318720219
ROUGE:  {'rouge-1': {'r': 0.029411764705882353, 'p': 0.029411764705882353, 'f': 0.02941176220588257}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.029411764705882353, 'p': 0.029411764705882353, 'f': 0.02941176220588257}}
Model: IlyaGusev/rut5_base_sum_gazeta
summary: В Кремле прошло праздничное шоу «Танцуют все!» на телеканале «Россия». Зрителям рассказали,

## 2. Сделаем генерацию заголовков для статьи, т.е. обучим модель для генерацию заголовков

In [9]:
df_test['title_clean'] = df_test['title'].progress_apply(lambda v: 'BOS ' + v + ' EOS')
df_train['title_clean'] = df_train['title'].progress_apply(lambda v: 'BOS ' + v + ' EOS')

100%|██████████| 52400/52400 [00:00<00:00, 759039.93it/s]


In [10]:
df_train = df_train[:MAX_TRAIN_SAMPLE]
df_test = df_test[:MAX_TEST_SAMPLE]

In [11]:
trans_2 = Transform_2()
trans_2.transfrom_text(df_train['text'], df_test['text'])

In [12]:
trans_2_sum = Transform_2(100)
trans_2_sum.transfrom_text(df_train['title_clean'], df_test['title_clean'])

In [13]:
reverse_text_index = trans_2.get_token().index_word
reverse_sum_index = trans_2_sum.get_token().index_word
sum_wordindex = trans_2_sum.get_token().word_index

### Строим модели энкодера и декодера

In [14]:
# Переопределяем буффер
BUFFER_SIZE = len(trans_2.get_padded()['train'])
# Переотпределяем шаг
steps_per_epoch = len(trans_2.get_padded()['train'])//BATCH_SIZE

# Каждую пару (padded_x_train, padded_x_train_sum) преобразуем в тензор
dataset = tf.data.Dataset.from_tensor_slices((trans_2.get_padded()['train'], trans_2_sum.get_padded()['train'])).shuffle(BUFFER_SIZE)
# Объединяем последоваетльные элементы в батчи, неполный батч удаляем
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [15]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 100]), TensorShape([64, 100]))

In [16]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru1(x, initial_state = hidden)
        output, state = self.gru2(output, initial_state = state)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
    
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = tf.keras.layers.AdditiveAttention()

    def call(self, x, query, value):
        # enc_output shape == (batch_size, max_length, hidden_size)
        #attention_weights = self.attention([ tf.expand_dims(query, 1), value,])
        context_vector = self.attention([tf.expand_dims(query, 1), value,])
        #context_vector = tf.squeeze(context_vector)


        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([context_vector, x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [17]:
latent_dim = 300
embedding_dim=200

# создаем экземпляры encoder и decoder
encoder = Encoder(trans_2.get_size(), embedding_dim, latent_dim, BATCH_SIZE)
decoder = Decoder(trans_2_sum.get_size(), embedding_dim, latent_dim, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

### Обучаем модели

In [18]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

checkpoint_dir = './training_summ_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [19]:
for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 10 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
    # saving (checkpoint) the model every 50 epochs
    if (epoch + 1) % 50 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for {} epoch {} sec\n'.format(epoch + 1, time.time() - start))

Epoch 1 Batch 0 Loss 0.6134
Epoch 1 Batch 10 Loss 0.5680
Epoch 1 Batch 20 Loss 0.5290
Epoch 1 Batch 30 Loss 0.5488
Epoch 1 Batch 40 Loss 0.5106
Epoch 1 Batch 50 Loss 0.5156
Epoch 1 Batch 60 Loss 0.5311
Epoch 1 Batch 70 Loss 0.4592
Epoch 1 Batch 80 Loss 0.5174
Epoch 1 Batch 90 Loss 0.4883
Epoch 1 Batch 100 Loss 0.4783
Epoch 1 Batch 110 Loss 0.4902
Epoch 1 Batch 120 Loss 0.5179
Epoch 1 Batch 130 Loss 0.4986
Epoch 1 Batch 140 Loss 0.4517
Epoch 1 Batch 150 Loss 0.5108
Epoch 1 Loss 0.5125
Time taken for 1 epoch 183.75846195220947 sec

Epoch 2 Batch 0 Loss 0.4600
Epoch 2 Batch 10 Loss 0.4766
Epoch 2 Batch 20 Loss 0.4633
Epoch 2 Batch 30 Loss 0.4564
Epoch 2 Batch 40 Loss 0.4267
Epoch 2 Batch 50 Loss 0.4724
Epoch 2 Batch 60 Loss 0.4515
Epoch 2 Batch 70 Loss 0.4936
Epoch 2 Batch 80 Loss 0.4899
Epoch 2 Batch 90 Loss 0.4396
Epoch 2 Batch 100 Loss 0.4691
Epoch 2 Batch 110 Loss 0.4636
Epoch 2 Batch 120 Loss 0.4357
Epoch 2 Batch 130 Loss 0.5025
Epoch 2 Batch 140 Loss 0.4717
Epoch 2 Batch 150 Loss 0.

### Оцениваем модель

In [20]:
for i in range(5):
  padded_test = trans_2_sum.get_padded()["test"][i]
  text2 = seq2text(padded_test)
  print(f"Original title: {seq2summary(padded_test)}\nPredicted title: {evaluate(text2.strip(), trans_2.get_token(), trans_2_sum.get_token())}\n" )

Original title: или куда полетит 
Predicted title: «мощнейшая» ракета будет порошенко eos 
Original title: «люди в бурятии очень 
Predicted title: «интер» не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в сша не будет в 
Original title: ссср как лукашенко провел выборы 
Predicted title: ягр будет в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сша маккейн в сш

# Вывод

1. При попытки использовать модель _IlyaGusev/rut5_base_sum_gazeta_  результат вышел не очень хороший, потому были попытки использовать более развернутая модель _IlyaGusev/mbart_ru_sum_gazeta_, но и она не дала хороших результатов.

2. Плохой результат... Походу, переобучение сказалось на модели. По логам обучения видно, что переобучение началось в 80-ых эпохах и в 100-ых эпохах окончательно сошлось все к нулю. По хорошему, надо брать меньше эпох и уменьшить размер бача, тогда может получиться более менее нормальная модель.